In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf

import splitfolders
input_folder = r"C:\Users\91900\OneDrive\Desktop\Final_SVM_Model"
splitfolders.ratio(input_folder, output = "SVM_Split_data", seed = 42, ratio = (.7, .2, .1), group_prefix = None) #Default values
splitfolders.fixed(input_folder, output = "SVM_Split_data", seed = 42, fixed = (35,20),oversample = False, group_prefix = None) #Default values

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
train_path =r"C:\Users\91900\OneDrive\Desktop\SVM_Split_data\train"
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import Sequential
from tensorflow import keras
from sklearn.model_selection import KFold

alphabet_labels = sorted(os.listdir(train_path))
print(alphabet_labels)
batch_size = 64
target_size = (48,48)

# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
training_set = train_datagen.flow_from_directory(r'C:\Users\91900\OneDrive\Desktop\SVM_Split_data\train',
target_size = (64, 64),
batch_size = 32,
class_mode = 'categorical')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(r'C:\Users\91900\OneDrive\Desktop\SVM_Split_data\test',
target_size = (64, 64),
batch_size = 32,
class_mode = 'categorical')

# Part 2 - Building the SVM
# Initialising the SVM
svm = tf.keras.models.Sequential()

# Step 1 - Convolution
svm.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', strides=2, input_shape=[64, 64, 3]))
# Step 2 - Pooling
svm.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# Adding a second convolutional layer
svm.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
svm.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# Step 3 - Flattening
svm.add(tf.keras.layers.Flatten())
# Step 4 - Full Connection
svm.add(tf.keras.layers.Dense(units=128, activation='relu'))
# Step 5 - Output Layer
#svm.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
## For Binary Classification
#svm.add(Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation='linear'))
## for mulitclassification
svm.add(Dense(4, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation
='softmax'))
svm.summary()

# Part 3 - Training the SVM
# Compiling the SVM
svm.compile(optimizer = 'adam', loss = 'squared_hinge', metrics = ['accuracy'])
# Training the SVM on the Training set and evaluating it on the Test set
r=svm.fit(x = training_set, validation_data = test_set, epochs = 15)

# plot the loss
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')
# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')
# save it as a h5 file
from tensorflow.keras.models import load_model
svm.save('model_svm.h5')
